In [1]:
import sys
sys.path.append('../')

In [2]:
#import my functions
import helpers.fileHandler as fh
import utils._modelDefinitions as _model
import utils._indexDefinitions as _index
import utils.sstIndex as sst
import utils.pslIndex as psl
import utils.timePeriod as tp
import utils.compound as compound


In [3]:
import xarray
import numpy
import matplotlib.pyplot as plt

In [4]:
deckSet=['piControl','historical']
scenarioSet=['ssp126', 'ssp245', 'ssp370','ssp585']
experimentSet=[*deckSet, *scenarioSet]

In [5]:
modelSet=_model.scenarioMip

In [6]:
sstIndeces = _index.sstIndex.keys()
pslIndeces = _index.pslIndex

In [8]:
for i in range(0,len(modelSet)):
    
    iModel=modelSet[i]
    print(i + iModel)
    
    for experiment in experimentSet: 
        
        if experiment=='piControl':
            variant = iModel[2]
        else:
            variant = iModel[3]

        try:
            tsDs = fh.loadModelData(iModel[1], 'ts_Amon', experiment, variant).ts.to_dataset()
            sstDs=tsDs.assign_attrs({'project_id':'CMIP'})

            sst=[
                sstDs.time.dt.year[0].values,
                sstDs.time.dt.year[-1].values
            ]

            pslDs = fh.loadModelData(iModel[1], 'psl_Amon', experiment,variant)

            psl=[
                pslDs.time.dt.year[0].values,
                pslDs.time.dt.year[-1].values
            ]

            indecesDs = xarray.open_dataset('../results/cmipWarmSeasonIndeces/' + iModel[1] +'_'+ experiment + '.nc')

            cal=[
                indecesDs.year[0].values, 
                indecesDs.year[-1].values
            ]
            
            if sst==psl:
                #'ssl & psl' match
                print(cal)
            
        except Exception as e:
            print(e)
        
        

['CSIRO-ARCCSS' 'ACCESS-CM2' 'r1i1p1f1' 'r1i1p1f1']
sst: 950 - 1449
psl: 950 - 1449
cal: 950 - 1448
sst: 1850 - 2014
psl: 1850 - 2014
cal: 1850 - 2013
sst: 2015 - 2300
psl: 2015 - 2300
cal: 1850 - 2299
sst: 2015 - 2100
psl: 2015 - 2100
cal: 1850 - 2099
sst: 2015 - 2100
psl: 2015 - 2100
cal: 1850 - 2099
sst: 2015 - 2300
psl: 2015 - 2300
cal: 1850 - 2299
['CSIRO' 'ACCESS-ESM1-5' 'r1i1p1f1' 'r1i1p1f1']
sst: 101 - 1100
psl: 101 - 1100
cal: 101 - 1099
sst: 1850 - 2014
psl: 1850 - 2014
cal: 1850 - 2013
sst: 2015 - 2300
psl: 2015 - 2300
cal: 1850 - 2299
sst: 2015 - 2100
psl: 2015 - 2100
cal: 1850 - 2099
sst: 2015 - 2100
psl: 2015 - 2100
cal: 1850 - 2099
sst: 2015 - 2300
psl: 2015 - 2300
cal: 1850 - 2299
['AWI' 'AWI-CM-1-1-MR' 'r1i1p1f1' 'r1i1p1f1']
sst: 2401 - 2900
psl: 2898 - 2900
cal: 2401 - 2899
sst: 1850 - 2014
psl: 1850 - 2014
cal: 1850 - 2013
sst: 2015 - 2100
psl: 2015 - 2100
cal: 1850 - 2099
sst: 2015 - 2100
psl: 2015 - 2100
cal: 1850 - 2099
sst: 2015 - 2100
psl: 2015 - 2100
cal: 1850 

/Users/ajs/opt/anaconda3/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'ts' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


sst: 1 - 1200


/Users/ajs/opt/anaconda3/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'psl' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


psl: 1 - 1200
cal: 1 - 1199


...

psl: 2015 - 2100
cal: 1850 - 2099


/Users/ajs/opt/anaconda3/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'ts' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


sst: 2015 - 2299


/Users/ajs/opt/anaconda3/lib/python3.8/site-packages/xarray/conventions.py:512: SerializationWarning: variable 'psl' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


psl: 2015 - 2299
cal: 1850 - 2298
['THU' 'CIESM' 'r1i1p1f1' 'r1i1p1f1']
sst: 1 - 500
psl: 1 - 500
cal: 1 - 499
sst: 1850 - 2014
psl: 1850 - 2014
cal: 1850 - 2013
sst: 2015 - 2100
psl: 2015 - 2100
cal: 1850 - 2099
sst: 2015 - 2100
psl: 2015 - 2100
cal: 1850 - 2099
CIESMts_Amonssp370r1i1p1f1 file not found on ESGF
Files (filter term: ts_Amon_.*?CIESM_ssp370_r1i1p1f1_.*?\.nc ) not found, possibly test name is wrong
sst: 2015 - 2100
psl: 2015 - 2100
cal: 1850 - 2099
['CMCC' 'CMCC-CM2-SR5' 'r1i1p1f1' 'r1i1p1f1']
sst: 1850 - 2349
psl: 1850 - 2349
cal: 1850 - 2348
sst: 1850 - 2014
psl: 1850 - 2014
cal: 1850 - 2013
sst: 2015 - 2100
psl: 2015 - 2100
cal: 1850 - 2099
sst: 2015 - 2100
psl: 2015 - 2100
cal: 1850 - 2099
sst: 2015 - 2100
psl: 2015 - 2100
cal: 1850 - 2099
sst: 2015 - 2100
psl: 2015 - 2100
cal: 1850 - 2099
['CMCC' 'CMCC-ESM2' 'r1i1p1f1' 'r1i1p1f1']
sst: 1850 - 2349
psl: 1850 - 2349
cal: 1850 - 2348
sst: 1850 - 2014
psl: 1850 - 2014
cal: 1850 - 2013
sst: 2015 - 2100
psl: 2015 - 2100
ca